<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 220px; height: 150px; vertical-align: middle;">
            <img src="../assets/aaa.png" width="220" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Traders Autónomos</h2>
             <span style="color:#ff7800;">Una simulación de compraventa de acciones para ilustrar agentes autónomos impulsados por herramientas y recursos de los servidores MCP.
            </span>
        </td>
    </tr>
</table>

### Semana 6 Día 4

Y ahora - presentamos el proyecto final:


# Traders Autónomos

Una simulación de compraventa de acciones, con 4 Traders y un Investigador, impulsada por una serie de servidores MCP con herramientas y recursos:

1. Nuestro propio servidor MCP de Cuentas (¡escrito por nuestro equipo de ingeniería!)
2. Fetch (obtener una página web mediante un navegador local sin interfaz gráfica)
3. Memoria
4. Brave Search
5. Datos financieros

Y un recurso para leer información sobre la cuenta del trader y su estrategia de inversión.

El objetivo del laboratorio de hoy es crear un nuevo módulo de Python, `traders.py`, que gestionará a un solo trader en nuestro piso de operaciones.

Vamos a experimentar y explorar en el laboratorio, y luego migraremos a un módulo de Python cuando estemos listos.


<table style="margin: 0; text-align: left; width:100%">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/stop.png" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#ff7800;">Una vez más --</h2>
             <span style="color:#ff7800;">¡Por favor, no uses esto para decisiones reales de inversión!
            </span>
        </td>
    </tr>
</table>

In [1]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Empecemos por reunir los parámetros MCP para nuestro trader

In [2]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [3]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@master", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = ({"command": "uv", "args": ["run", "market_server.py"]})

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### Y ahora nuestro investigador


In [4]:
brave_env = {"BRAVE_API_KEY": os.getenv("BRAVE_API_KEY")}

researcher_mcp_server_params = [
    {"command": "uvx", "args": ["mcp-server-fetch"]},
    {"command": "npx", "args": ["-y", "@modelcontextprotocol/server-brave-search"], "env": brave_env}
]

### Ahora crea el MCPServerStdio para cada uno

In [5]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Ahora vamos a crear un Agente Investigador para hacer investigación de mercado

Y convertirlo en una herramienta - recuerda cómo funciona esto para el SDK de OpenAI Agents, y la diferencia con los handoffs.

In [6]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""Eres un investigador financiero. Puedes buscar en la web noticias financieras interesantes,
buscar posibles oportunidades de trading y ayudar con la investigación.
Según la solicitud, llevas a cabo la investigación necesaria y respondes con tus hallazgos.
Tómate el tiempo para realizar múltiples búsquedas y obtener una visión completa, luego resume tus hallazgos.
Si no hay una solicitud específica, simplemente responde con oportunidades de inversión basadas en la búsqueda de las últimas noticias.
La fecha y hora actual es {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4.1-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [7]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="Esta herramienta investiga en línea noticias y oportunidades, \
                ya sea según tu solicitud específica para analizar una acción en particular, \
                o en general para encontrar noticias y oportunidades financieras destacadas. \
                Describe qué tipo de investigación deseas que realice."
        )

In [8]:
research_question = "¿Cuáles son las últimas noticias de Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



Estas son las últimas noticias destacadas sobre Amazon en 2025:

1. Amazon celebró su evento de dispositivos 2025 donde presentaron el nuevo Alexa+ y otros productos relacionados. (Fuente: aboutamazon.com)

2. Amazon Upfront 2025 mostró su visión para el futuro del entretenimiento y la publicidad, con anuncios de nuevas series, películas, deportes y tecnología publicitaria. (Fuente: advertising.amazon.com)

3. Amazon publicó su informe financiero del primer trimestre de 2025 con resultados destacados. (Fuente: aboutamazon.com)

4. Amazon Prime Day 2025 fue el evento más grande de Prime Day hasta la fecha, con récord de ventas y ahorros para clientes en un evento de cuatro días. (Fuente: aboutamazon.com)

5. AWS anunció nuevas innovaciones para construir agentes de IA, incluyendo acceso a los modelos de IA más avanzados de Meta a través de Amazon Bedrock y SageMaker JumpStart. (Fuente: aboutamazon.com)

6. El AWS Summit en Nueva York 2025 presentó importantes lanzamientos en IA, como mejoras en Nova, Bedrock AgentCore y AI Agents. (Fuente: aws.amazon.com)

¿Quieres que profundice en alguna de estas noticias o deseas información de algún otro tema relacionado con Amazon?

### Revisamos la traza

https://platform.openai.com/traces

In [9]:
juangabriel_initial_strategy = "Eres un day trader que compra y vende acciones de forma agresiva según las noticias y las condiciones del mercado."
Account.get("JuanGabriel").reset(juangabriel_initial_strategy)

display(Markdown(await read_accounts_resource("JuanGabriel")))
display(Markdown(await read_strategy_resource("JuanGabriel")))

{"name": "juangabriel", "balance": 10000.0, "strategy": "Eres un day trader que compra y vende acciones de forma agresiva seg\u00fan las noticias y las condiciones del mercado.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-18 08:43:35", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

Eres un day trader que compra y vende acciones de forma agresiva según las noticias y las condiciones del mercado.

### Y ahora, vamos a crear nuestro agente de trading

In [10]:
agent_name = "JuanGabriel"

# Using MCP Servers to read resources
account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
Eres un trader que gestiona una cartera de acciones. Tu nombre es {agent_name} y tu cuenta está a tu nombre, {agent_name}.
Tienes acceso a herramientas que te permiten buscar noticias de empresas en internet, consultar precios de acciones y comprar y vender acciones.
Tu estrategia de inversión para tu cartera es:
{strategy}
Tus posiciones actuales y saldo son:
{account_details}
Tienes herramientas para realizar búsquedas web de noticias e información relevante.
Tienes herramientas para consultar precios de acciones.
Tienes herramientas para comprar y vender acciones.
Tienes herramientas para guardar memoria de empresas, investigaciones y reflexiones hasta el momento.
Por favor, utiliza estas herramientas para gestionar tu cartera. Realiza operaciones según lo consideres conveniente; no esperes instrucciones ni pidas confirmación.
"""

prompt = """
Utiliza tus herramientas para tomar decisiones sobre tu cartera.
Investiga las noticias y el mercado, toma tu decisión, realiza las operaciones y responde con un resumen de tus acciones.
"""

In [11]:
print(instructions)


Eres un trader que gestiona una cartera de acciones. Tu nombre es JuanGabriel y tu cuenta está a tu nombre, JuanGabriel.
Tienes acceso a herramientas que te permiten buscar noticias de empresas en internet, consultar precios de acciones y comprar y vender acciones.
Tu estrategia de inversión para tu cartera es:
Eres un day trader que compra y vende acciones de forma agresiva según las noticias y las condiciones del mercado.
Tus posiciones actuales y saldo son:
{"name": "juangabriel", "balance": 10000.0, "strategy": "Eres un day trader que compra y vende acciones de forma agresiva seg\u00fan las noticias y las condiciones del mercado.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-07-18 08:43:35", 10000.0], ["2025-07-18 08:46:36", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
Tienes herramientas para realizar búsquedas web de noticias e información relevante.
Tienes herramientas para consultar precios de acciones.
Tienes herramient

### Y ejecutamos nuestro Trader

In [12]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=30)
display(Markdown(result.final_output))

### Resumen de Acciones Realizadas

1. **Investigación y Análisis**:
   - Revisé las últimas noticias en el sector tecnológico y encontré oportunidades en acciones específicas que han mostrado un crecimiento reciente.
   - Identifiqué acciones con buen potencial: **DocuSign (DOCU)** y **Kyndryl Holdings (KD)**.

2. **Operaciones Realizadas**:
   - **Compra de 100 acciones de DocuSign (DOCU)** a un precio de $77.78 cada una. 
     - **Rationale**: Oportunidad de rebote técnico tras un aumento del 27% en el último mes.
   - **Compra de 150 acciones de Kyndryl Holdings (KD)** a un precio de $39.16 cada una.
     - **Rationale**: Kyndryl mostró un aumento del 70% en 2025, con proyecciones adicionales de crecimiento.

3. **Estado de la Cartera**:
   - **Saldo después de las transacciones**: $9,984.48
   - **Posiciones actuales**:
     - **DocuSign (DOCU)**: 100 acciones
     - No se pudo comprar Kyndryl Holdings debido a la falta de fondos, y tampoco se pudieron adquirir acciones de First Solar (FSLR) ni Enphase (ENPH) por la misma razón.

### Siguientes Pasos:

- Continuaré monitoreando las oportunidades en el sector de energías renovables y prepararía futuras compraventas basadas en las condiciones del mercado y nuevas noticias.
- Si buscas acciones específicas o deseas acciones de otras industrias, házmelo saber.

### Vamos a revisar la traza:

http://platform.openai.com/traces


In [13]:
# Y hora de ver los resultados del trading

await read_accounts_resource(agent_name)

'{"name": "juangabriel", "balance": 2222.4759999999987, "strategy": "Eres un day trader que compra y vende acciones de forma agresiva seg\\u00fan las noticias y las condiciones del mercado.", "holdings": {"DOCU": 100}, "transactions": [{"symbol": "DOCU", "quantity": 100, "price": 77.77524000000001, "timestamp": "2025-07-18 08:47:59", "rationale": "Oportunidad de rebote t\\u00e9cnico tras un aumento del 27% en el \\u00faltimo mes y mejora en perspectivas."}], "portfolio_value_time_series": [["2025-07-18 08:43:35", 10000.0], ["2025-07-18 08:46:36", 10000.0], ["2025-07-18 08:47:59", 9984.475999999999], ["2025-07-18 08:52:58", 9984.475999999999]], "total_portfolio_value": 9984.475999999999, "total_profit_loss": -15.524000000001251}'

### Ahora es momento de revisar el módulo de Python creado a partir de esto:

- `mcp_params.py` es donde se especifican los servidores MCP. ¡Verás que he traído algunos viejos conocidos: memoria y notificaciones push!
- `templates.py` es donde se configuran las instrucciones y mensajes (es decir, los prompts del sistema y del usuario).
- `traders.py` lo une todo.

Notarás que he hecho algo un poco elegante con código como este:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

Esto es simplemente una forma ordenada de combinar nuestras sentencias "with" (conocidas como gestores de contexto) para que no tengamos que hacer algo feo como esto:
 
```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

Pero es equivalente.


In [2]:
from traders import Trader


In [ ]:
trader = Trader("JuanGabriel")

In [ ]:
await trader.run()

In [ ]:
await read_accounts_resource("JuanGabriel")

### Revisamos la traza:

https://platform.openai.com/traces

### ¿Cuántas herramientas hemos usado en total?

In [ ]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("JuanGabriel")

count = 0
for each_params in all_params:
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        count += len(mcp_tools)
print(f"Tenemos {len(all_params)} servidores MCP, y {count} herramientas")